In [1]:
import sys
# sys.path.append('./keras-spp/')
# from spp.SpatialPyramidPooling import SpatialPyramidPooling

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import glob
import os
from scipy.misc import imread, imresize
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgb2gray
from keras.utils import np_utils
import pywt
import matplotlib.pyplot as plt
import cv2
from skimage.transform import resize


Using TensorFlow backend.


#### Load Data

In [18]:
master_folder = '../../Desktop/PhotosDataset'

imgs = []
labels = []
for photo in os.listdir(master_folder+'/Alex'):
    img = imread(master_folder+'/Alex/'+photo)
    imgs.append(resize(img,(200,200,3)))
    labels.append('Alex')
for photo in os.listdir(master_folder+'/Hunter'):
    img = imread(master_folder+'/Hunter/'+photo)
    imgs.append(resize(img,(200,200,3)))
    labels.append('Hunter')
labels = np.array([[1,0] if label is 'Hunter' else [0,1] for label in labels])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.


#### Massage Data

In [20]:
# Take out alpha component of image
#imgs = [img[:,:,[0,1,2]] for img in imgs]

wavelet_imgs = []
for i, img in enumerate(imgs):
    if i % 50 == 0:
        print(i)
    coeffs2 = pywt.dwt2(img, 'bior1.3')
    wavelet_imgs.append(coeffs2[1][:2])

for i,stack in enumerate(wavelet_imgs):
    stack = np.dstack(stack)
    wavelet_imgs[i] = stack


x_train, x_test, y_train, y_test = train_test_split(wavelet_imgs, labels, test_size=.1)
x_train = np.stack(x_train,axis=0)
x_test = np.stack(x_test,axis=0)

num_classes = 2
# y_train = np_utils.to_categorical(y_train, num_classes)
# y_test = np_utils.to_categorical(y_test, num_classes)


0
50
100
150
200
250
300
350


In [16]:
# x_train = np.stack(x_train,axis=0)
# x_test = np.stack(x_test,axis=0)

num_classes = 2
# y_train = np_utils.to_categorical(y_train, num_classes)
# y_test = np_utils.to_categorical(y_test, num_classes)

In [21]:
# # Use a generator to feed data because data images are of different dimensions so a numpy array cant
# # be constructed
# def generator(x, y):
#     while True:
#         for i,img in enumerate(x):
#             yield np.expand_dims(img,axis=0), np.expand_dims(y[i],axis=0)
            

            
# gen = generator(x_train, y_train)
# val_gen = generator(x_test,y_test)
num_channels = 2
# num_classes = 2
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

datagen.fit(x_train)
# gen = generator(x_train, y_train)
# val_gen = generator(x_test,y_test)

batch_size=4
epochs=10


In [22]:
print(y_train.shape)

(332, 2)


### Model with no transfer learning

In [23]:
model = Sequential()

# Note that we leave the image size as None to allow multiple image sizes
model.add(Conv2D(96, (7, 7),strides=2, padding='same',
                 activation='relu',input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]),
                name='input_layer'))
model.add(MaxPooling2D((3,3)))

model.add(Conv2D(48, (5, 5),strides=1, padding='same',activation='relu'))
model.add(MaxPooling2D((3,3)))

model.add(Conv2D(24, (3, 3),strides=1, padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(24, (3, 3),strides=1, padding='same',activation='relu'))
model.add(Flatten())
# # Spatial Pooling layer to deal with differing image sizes
# model.add(SpatialPyramidPooling([6, 3, 2,1]))

model.add(Dense(128,activation='relu'))

# Classification layer
model.add(Dense(2, activation='softmax'))

callbacks = []
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                   patience=15, verbose=1, mode='auto',
                                   min_delta=0.0001, cooldown=0, min_lr=0))
# opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


hist = model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),verbose=1,
                    callbacks=callbacks)




/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras_preprocessing/image/numpy_array_iterator.py:127: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (332, 200, 102, 8) (8 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/10
83/83 [==============================] - 7s 81ms/step - loss: 0.6999 - acc: 0.5753 - val_loss: 0.6651 - val_acc: 0.8649
Epoch 2/10
83/83 [==============================] - 6s 76ms/step - loss: 0.2230 - acc: 0.9337 - val_loss: 0.4876 - val_acc: 0.9189
Epoch 3/10
83/83 [==============================] - 7s 80ms/step - loss: 0.0529 - acc: 0.9849 - val_loss: 0.5049 - val_acc: 0.8649
Epoch 4/10
83/83 [==============================] - 7s 78ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.4672 - val_acc: 0.8649
Epoch 5/10
83/83 [==============================] - 6s 77ms/step - loss: 3.4346e-04 - acc: 1.0000 - val_loss: 0.4423 - val_acc: 0.8649
Epoch 6/10
83/83 [==============================] - 6s 77ms/step - loss: 2.4497e-04 - acc: 1.0000 - val_loss: 0.4251 - val_acc: 0.8649
Epoch 7/10
83/83 [==============================] - 7s 81ms/step - loss: 0.0920 - acc: 0.9699 - val_loss: 0.5231 - val_acc: 0.8649
Epoch 8/10
83/83 [==============================] - 7s 81ms/step - loss: 7.

### Utilize transfer learning

In [24]:
filepath = "wavelet_model.h5"
trans_model = load_model(filepath)

# Allow no weight adjustments for the pretrained layers
for layer in trans_model.layers:
    layer.trainable = False

trans_model.add(Conv2D(32,(2,2)))
trans_model.add(MaxPooling2D((1,1)))
trans_model.add(Flatten())
trans_model.add(Dense(128,activation='relu'))

# Classification Layer
trans_model.add(Dense(num_classes, activation='softmax'))

W1130 18:03:41.065224 140734961169856 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [25]:
callbacks = []
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                   patience=15, verbose=1, mode='auto',
                                   min_delta=0.0001, cooldown=0, min_lr=0))
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


epochs = 10
# Fit model
trans_hist = trans_model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),verbose=1,
                    callbacks=callbacks)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras_preprocessing/image/numpy_array_iterator.py:127: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (332, 200, 102, 8) (8 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/10
83/83 [==============================] - 9s 103ms/step - loss: 7.9252 - acc: 0.5030 - val_loss: 10.4550 - val_acc: 0.3514
Epoch 2/10
83/83 [==============================] - 8s 91ms/step - loss: 8.0105 - acc: 0.5030 - val_loss: 10.4550 - val_acc: 0.3514
Epoch 3/10
83/83 [==============================] - 7s 90ms/step - loss: 8.0105 - acc: 0.5030 - val_loss: 10.4550 - val_acc: 0.3514
Epoch 4/10
83/83 [==============================] - 8s 92ms/step - loss: 8.0105 - acc: 0.5030 - val_loss: 10.4550 - val_acc: 0.3514
Epoch 5/10
83/83 [==============================] - 8s 98ms/step - loss: 8.0105 - acc: 0.5030 - val_loss: 10.4550 - val_acc: 0.3514
Epoch 6/10
83/83 [==============================] - 9s 102ms/step - loss: 8.0105 - acc: 0.5030 - val_loss: 10.4550 - val_acc: 0.3514
Epoch 7/10
83/83 [==============================] - 8s 96ms/step - loss: 8.0105 - acc: 0.5030 - val_loss: 10.4550 - val_acc: 0.3514
Epoch 8/10
83/83 [==============================] - 9s 107ms/step - loss: 

Evaluate

In [26]:
trans_pred = []
reg_pred= []
for img in x_test:
    trans_pred.append(trans_model.predict(np.expand_dims(img, axis=0)))
    reg_pred.append(model.predict(np.expand_dims(img, axis=0)))

reg_pred = ["Hunter" if x[0][0] > x[0][1] else "Alex" for x in reg_pred]
trans_pred = ["Hunter" if x[0][0] > x[0][1] else "Alex" for x in trans_pred]

y_test = ["Hunter" if x[0] > x[1] else "Alex" for x in y_test]

In [27]:
from sklearn.metrics import classification_report
print("Model with No transfer learning")
print(classification_report(y_test, reg_pred))
print("Model with transfer learning")
#print(classification_report(y_test, trans_pred))

Model with No transfer learning
              precision    recall  f1-score   support

        Alex       0.72      1.00      0.84        13
      Hunter       1.00      0.79      0.88        24

    accuracy                           0.86        37
   macro avg       0.86      0.90      0.86        37
weighted avg       0.90      0.86      0.87        37

Model with transfer learning
